- Ref: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen3_(4B)-GRPO.ipynb#scrollTo=DkIvEkIIkEyB

Instal unsloth

In [1]:
import torch
print(torch.__version__)
# uv add "unsloth[cu124-torch251] @ git+https://github.com/unslothai/unsloth.git"

2.6.0+cu124


Pre-fine-tune the model to make GRPO follow the desired output format — this speeds up GRPO.

In [2]:
import unsloth
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/sean/MLAlgoFromScratch/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 09-05 04:00:24 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 09-05 04:00:24 [__init__.py:239] Automatically detected platform cuda.


2025-09-05 04:00:26,739	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset, load_dataset

from trl import SFTTrainer, SFTConfig

In [4]:
max_seq_length = 2048

In [ ]:
# key: vllm==0.8.5.post1
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='unsloth/Qwen3-4B-base',
    max_seq_length=512,
    load_in_4bit=True,
    fast_inference=True, # Enable vLLM fast inference
    max_lora_rank=8,
    gpu_memory_utilization=0.6 # Reduce if OOM
)

Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.7.8: Fast Qwen3 patching. Transformers: 4.54.0. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.581 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen3-4b-base-unsloth-bnb-4bit with actual GPU utilization = 59.0%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.58 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 5.94 GB. Also swap space = 0 GB.
WARNING 09-05 04:00:39 [config.py:2972] Casting torch.bfloat16 to torch.float16.
INFO 09-05 04:00:48 [config.py:71

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.53s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.53s/it]

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.40s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.40s/it]


INFO 09-05 04:00:58 [punica_selector.py:18] Using PunicaWrapperGPU.


INFO 09-05 04:00:58 [model_runner.py:1140] Model loading took 3.1446 GiB and 6.297541 seconds
INFO 09-05 04:01:02 [worker.py:287] Memory profiling takes 3.77 seconds
INFO 09-05 04:01:02 [worker.py:287] the current vLLM instance can use total_gpu_memory (14.58GiB) x gpu_memory_utilization (0.59) = 8.60GiB
INFO 09-05 04:01:02 [worker.py:287] model weights take 3.14GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 1.05GiB; the rest of the memory reserved for KV Cache is 4.39GiB.
INFO 09-05 04:01:02 [executor_base.py:112] # cuda blocks: 1995, # CPU blocks: 0
INFO 09-05 04:01:02 [executor_base.py:117] Maximum concurrency for 512 tokens per request: 62.34x
INFO 09-05 04:01:02 [vllm_utils.py:669] Unsloth: Running patched vLLM v0 `capture_model`.
INFO 09-05 04:01:02 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager

Capturing CUDA graph shapes: 100%|██████████| 27/27 [00:14<00:00,  1.87it/s]

INFO 09-05 04:01:17 [model_runner.py:1592] Graph capturing finished in 14 secs, took 4.24 GiB
INFO 09-05 04:01:17 [vllm_utils.py:676] Unsloth: Patched vLLM v0 graph capture finished in 14 secs.


INFO 09-05 04:01:18 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 19.73 seconds
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'post_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'post_feedforward_layernorm']


In [6]:
max_dim = tokenizer.model_max_length
print(f"Tokenizer can handle up to {max_dim} tokens.")

Tokenizer can handle up to 32768 tokens.


In [7]:
tokenizer.chat_template is None
# ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! 

True

In [8]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560, padding_idx=151654)
    (layers): ModuleList(
      (0-1): 2 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2560, out_features=4096, bias=False)
          (k_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=2560, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (up_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (down_proj): Linear(in_features=9728, out_features=2560, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RM

In [9]:
lora_rank = 8  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

Unsloth 2025.7.8 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [10]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 2560, padding_idx=151654)
        (layers): ModuleList(
          (0-1): 2 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2560, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
  

In [11]:
reasoning_start = "<start_working_out>" # Acts as <think>
reasoning_end   = "<end_working_out>"   # Acts as </think>
solution_start  = "<SOLUTION>"
solution_end    = "</SOLUTION>"

system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your solution between {solution_start}{solution_end}"""
system_prompt

'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>'

In [12]:
"""
- Always includes a system prompt (either from messages or a default one).
- Marks assistant turns with an eos_token so the model knows where each reply ends.
- Optionally adds a “start reasoning” marker to signal when the model should generate output.
- Uses placeholders that get replaced at runtime so you can change system prompts or special tokens without editing the Jinja template.
"""

chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

# Replace without specific template:
chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{reasoning_start}'")
tokenizer.chat_template = chat_template

In [ ]:
# Turn a structured list of messages into the exact string prompt the model will see.
tokenizer.apply_chat_template([
    {"role" : "assistant", "content" : f"{reasoning_start}I think it's 2.{reasoning_end}{solution_start}2{solution_end}"},
    {"role" : "user", "content" : "What is 2+2?"},
], tokenize = False, add_generation_prompt = True)

"You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION><|endoftext|>What is 1+1?<start_working_out>I think it's 2.<end_working_out><SOLUTION>2</SOLUTION><|endoftext|>What is 2+2?<start_working_out>"

In [14]:
# load dataset
dataset = load_dataset("unsloth/OpenMathReasoning-mini", split = "cot")
# dataset.to_pandas()

In [15]:
# use few sample of quick sanity check
# dataset = dataset[:2]
# dataset = Dataset.from_dict(dataset)

Note
- I think the reason of using math here is the answer is precise
- If the result is text, we may compute the text similarity to confirm the answer ?!

In [16]:
dataset = dataset.to_pandas()[
    ["expected_answer", "problem", "generated_solution"]
]

# Try converting to number - if not, replace with NaN
is_number = pd.to_numeric(pd.Series(dataset["expected_answer"]), errors = "coerce").notnull()
# Select only numbers
dataset = dataset.iloc[np.where(is_number)[0]]

In [17]:
def format_dataset(x):
    expected_answer = x["expected_answer"]
    problem = x["problem"]

    # Remove generated <think> and </think>
    thoughts = x["generated_solution"]
    thoughts = thoughts.replace("<think>", "").replace("</think>", "")

    # Strip newlines on left and right
    thoughts = thoughts.strip()
    # Add our custom formatting
    final_prompt = \
        reasoning_start + thoughts + reasoning_end + \
        solution_start + expected_answer + solution_end
    return [
        {"role" : "system",    "content" : system_prompt},
        {"role" : "user",      "content" : problem},
        {"role" : "assistant", "content" : final_prompt},
    ]

dataset["Messages"] = dataset.apply(format_dataset, axis = 1)

In [18]:
test_msg = dataset.iloc[0].Messages
test_msg

[{'role': 'system',
  'content': 'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>'},
 {'role': 'user',
  'content': 'Given $\\sqrt{x^2+165}-\\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.'},
 {'role': 'assistant',
  'content': "<start_working_out>Okay, let's see. I need to solve the equation √(x² + 165) - √(x² - 52) = 7, and find all positive values of x. Hmm, radicals can be tricky, but maybe if I can eliminate the square roots by squaring both sides. Let me try that.\n\nFirst, let me write down the equation again to make sure I have it right:\n\n√(x² + 165) - √(x² - 52) = 7.\n\nOkay, so the idea is to isolate one of the radicals and then square both sides. Let me try moving the second radical to the other side:\n\n√(x² + 165) = 7 + √(x² - 52).\n\nNow, if I square both sides, maybe I can get rid of the square ro

Check to see if it worked

In [19]:
# default of tokenize is True
tokenizer.apply_chat_template(test_msg, tokenize=False)

"You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION><|endoftext|>Given $\\sqrt{x^2+165}-\\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.<start_working_out>Okay, let's see. I need to solve the equation √(x² + 165) - √(x² - 52) = 7, and find all positive values of x. Hmm, radicals can be tricky, but maybe if I can eliminate the square roots by squaring both sides. Let me try that.\n\nFirst, let me write down the equation again to make sure I have it right:\n\n√(x² + 165) - √(x² - 52) = 7.\n\nOkay, so the idea is to isolate one of the radicals and then square both sides. Let me try moving the second radical to the other side:\n\n√(x² + 165) = 7 + √(x² - 52).\n\nNow, if I square both sides, maybe I can get rid of the square roots. Let's do that:\n\n(√(x² + 165))² = (7 + √(x² - 52))².\n\nSimplifying the left side:\n\nx² + 

In [20]:
tokenized_result = tokenizer.apply_chat_template(test_msg)
print(f"Tokenized result: {tokenized_result[:10]}... \nwith dimension: {len(tokenized_result)}")

Tokenized result: [2610, 525, 2661, 264, 3491, 624, 38687, 911, 279, 3491]... 
with dimension: 1682


Limit sequence length to max_seq_length / 2 during pre-fine-tuning to avoid overly long reasoning.

In [29]:
dataset = dataset.loc[[len(tokenizer.apply_chat_template(x)) < max_seq_length/2 for x in dataset['Messages']]]
dataset.shape

(58, 5)

Tokenize data and convert it to a HF compatible dataset format

In [30]:
dataset['text'] = tokenizer.apply_chat_template(dataset['Messages'].values.tolist(), tokenize=False)

/tmp/ipykernel_382184/1368114567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['text'] = tokenizer.apply_chat_template(dataset['Messages'].values.tolist(), tokenize=False)


In [32]:
dataset = Dataset.from_pandas(dataset)

Pre fine-tune the model so it follows the custom GRPO formatting

In [23]:
# check the result before pre fine-tune
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)
text = "What is the sqrt of 101?"
output = model.fast_generate(
    [text], 
    sampling_params=sampling_params, 
    lora_request=None
)
print(output[0].outputs[0].text)


Processed prompts: 100%|██████████| 1/1 [00:10<00:00, 10.02s/it, est. speed input: 1.00 toks/s, output: 27.44 toks/s]

 - Quora
What is the sqrt of 101?
6 Answers
Best
Gaurav Bakliwal
Answered 2 years ago · Author has 526 answers and 384.9K answer author has 20.8K answers and 10M answer views 2 y
What is the sqrt of 101?
First of all 101 is not a square. It is pretty close to 100 which is 10×10. However 101 is 4 more than 100, so take √(100+4)
=(√100)+√4
=10 + 2
100.50.00.25.00…
√101 =10.05
Related
What is the square root of 614? (in square root symbol)
What is the square root of 614? (in square root symbol)
2,08.8
614 = 4 *153.5
2,√153.5
614 is a prime number.
Related Question
What is the square root of 614? (in squared symbol)
2,20


In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args = SFTConfig(
        dataset_text_field='text',
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8, # update after accumulation step
        warmup_steps=5,
        num_train_epochs=2,
        learning_rate=2e-4,
        logging_steps=5,
        optim='adamw_8bit',
        weight_decay=0.01,
        lr_scheduler_type='linear',
        seed=3407,
        report_to='none'
    )

)

Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 58/58 [00:01<00:00, 49.51 examples/s]


In [34]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 58 | Num Epochs = 2 | Total steps = 16
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 16,515,072 of 4,038,983,168 (0.41% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,1.028100
10,0.865000
15,0.650900


TrainOutput(global_step=16, training_loss=0.8443804010748863, metrics={'train_runtime': 116.2844, 'train_samples_per_second': 0.998, 'train_steps_per_second': 0.138, 'total_flos': 733895643365376.0, 'train_loss': 0.8443804010748863})

In [35]:
text = tokenizer.apply_chat_template(
    dataset[0]["Messages"][:2],
    tokenize=False,
    add_generation_prompt=True # add this then the ending is "<start_working_out>"
)
text

'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION><|endoftext|>Jenifer has 82 cents in pennies and nickels. Her younger brother mistook all her nickels for dimes and counted the total as $1.47. How many pennies does Jenifer have?<start_working_out>'

In [36]:
# from transformers import TextStreamer
# _ = model.generate(
#     **tokenizer(text, return_tensors = "pt").to("cuda"),
#     temperature = 0,
#     max_new_tokens = 1024,
#     streamer = TextStreamer(tokenizer, skip_prompt = False),
# )

In [37]:
# clean memory
del dataset
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [38]:
dataset = load_dataset("open-r1/DAPO-Math-17k-Processed", "en", split = "train")
dataset

Dataset({
    features: ['prompt', 'solution', 'data_source', 'source_prompt', 'ability', 'reward_model', 'extra_info'],
    num_rows: 14116
})

In [39]:
# used few sample for quick sanity check
# dataset = dataset[:2]
# dataset = Dataset.from_dict(dataset)

In [40]:
# the problem
print(dataset[0]["prompt"])

In triangle $ABC$, $\sin \angle A = \frac{4}{5}$ and $\angle A < 90^\circ$. Let $D$ be a point outside triangle $ABC$ such that $\angle BAD = \angle DAC$ and $\angle BDC = 90^\circ$. Suppose that $AD = 1$ and that $\frac{BD}{CD} = \frac{3}{2}$. If $AB + AC$ can be expressed in the form $\frac{a\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$.


Synthesis the dataset with customized system prompt and quetion(the name is prompt in the dataset)

In [41]:
dataset = dataset.map(lambda x: {
    "prompt": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": x["prompt"]}
    ],
    "answer": x['solution']
})

Extract the final answer from a structured text that contains both:
- a reasoning section (e.g., <REASONING>...</REASONING>)
- and a solution section (e.g., <SOLUTION>...</SOLUTION>)

In [42]:
import re

# Add optional EOS token matching
solution_end_regex = r"</SOLUTION>[\s]{0,}" + \
    "(?:" + re.escape(tokenizer.eos_token) + ")?"

match_format = re.compile(
    rf"{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end_regex}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)
match_format

re.compile(r'<end_working_out>.*?<SOLUTION>(.+?)</SOLUTION>[\s]{0,}(?:<\|endoftext\|>)?[\s]{0,}$',
           re.MULTILINE|re.DOTALL|re.UNICODE)

In [43]:
# test case
match_format.findall(
    "Let me think!<end_working_out>"\
    f"<SOLUTION>\n2\n</SOLUTION>",
)

['\n2\n']

In [44]:
match_format.findall(
    "Let me think!<end_working_out>"\
    f"<SOLUTION>364</SOLUTION>",
)

['364']

Create reward function to match the format 
- +3 points if the output matches the full format exactly
- Partial reward if key format tags (<REASONING>, </REASONING>, etc.) are present (each adds +0.5).
- Penalty (-1.0) if a required tag is missing or appears more than once.

In [45]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

In [46]:
def match_format_partially(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!

        # No need to reward <start_working_out> since we always prepend it!
        # score += 0.5 if response.count(reasoning_start) == 1 else -1.0
        score += 0.5 if response.count(reasoning_end)   == 1 else -1.0
        score += 0.5 if response.count(solution_start)  == 1 else -1.0
        score += 0.5 if response.count(solution_end)    == 1 else -1.0
        scores.append(score)
    return scores

Reward of penalize the generated content

In [47]:
def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]['content']
    responses = [completion[0]['content'] for completion in completions]

    extracted_responses = [
        result.group(1)
        if (result := match_format.search(r)) is not None else None for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(-2.0) # Note that it is append to scores not score, which means no answer=-2, then next run
            continue
        
        # 5 points for correct answer
        if guess  == true_answer:
            score += 5.0

        # Match if spaces are seen but less reward
        elif guess.strip() == true_answer.strip():
            score += 3.5
        
        else:
            # Reward the answer if it is close via ratios
            try:
                ratio = float(guess) / float(true_answer)
                if ratio >= 0.9 and ratio <= 1.1:
                    score += 2.0
                elif ratio >= 0.8 and ratio <= 1.2:
                    score += 1.5
                else:
                    score += -2.5
            except:
                score += -4.5
        scores.append(score)

    return scores

Remove commas such as 123,456 or extract the number in a sentence for example "The solution is $20" -> 20

In [48]:
# Match numbers like 17, 123,456, 0.34, -0.234, 1,234,567.89
match_numbers = re.compile(r"-?(?:\d{1,3}(?:,\d{3})+|\d+)(?:\.\d+)?", re.MULTILINE | re.DOTALL)

def find_numbers_clean(text):
    """Return numbers as strings with commas removed."""
    return [n.replace(",", "") for n in match_numbers.findall(text)]

# If you'd like floats instead, use this:
def find_numbers_float(text):
    return [float(n.replace(",", "")) for n in match_numbers.findall(text)]

# Tests
print(find_numbers_clean("<SOLUTION>  0.34  </SOLUTION>"))      # ['0.34']
print(find_numbers_clean("<SOLUTION>  123,456  </SOLUTION>"))   # ['123456']
print(find_numbers_clean("<SOLUTION>  -0.234  </SOLUTION>"))    # ['-0.234']
print(find_numbers_clean("<SOLUTION> The solution is 17  </SOLUTION>"))  # ['17']

['0.34']
['123456']
['-0.234']
['17']


In [49]:
global PRINTED_TIMES
PRINTED_TIMES = 0 # a counter that increments every time check_numbers is called.
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5 # ets the frequency (here 5).  

In [75]:
def check_numbers(prompts, completions, answer, **kwargs):
    """
    Evaluate numeric answers extracted from model completions against ground truth.

    Scoring (per item):
      - 3.5  if the extracted numeric guess equals the true answer (after stripping & removing commas)
      - -1.5 if a number was extracted but it does not equal the true answer
      - -2.5 if no number could be extracted
      - 0    if parsing fails unexpectedly

    Args:
        prompts:
            A sequence of message lists. This function reads the question text from
            `prompts[0][-1]["content"]`
            Example shape: `[[{"role":"user","content":"..."}], ...]`
        completions:
            A sequence of generation lists. Each item should be a list whose first element
            has a `"content"` string, accessed as `completion[0]["content"]`.
            Example shape: `[[{"role":"assistant","content":"..."}], ...]`
        answers:
            Sequence of ground-truth answers (strings or numbers). Must align 1:1 with `completions`.

    Returns:
        List[float]: Scores aligned with `completions` / `answers`.

    Notes:
        - Compares numeric equality after normalizing commas in guesses.
        - `answers` are cast to float via `float(str(answer).strip())`.
    """

    question = prompts[0][-1]['content'] # prompts are duplicate based on num_generation, so we only select one of tem
    responses = [completion[0]["content"] for completion in completions] 


    # apply reg.search to extract the answer, since the completion may contain other string.
    extracted_responses = [
        guess.group(1) if (guess := match_numbers.search(r)) is not None else None for r in responses
    ]
    
    scores = []
    # Print only every few steps
    global PRINTED_TIMES
    global PRINT_EVERY_STEPS
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        print(
            '*'*20 + f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}"
        )
    PRINTED_TIMES += 1

    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(-2.5)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            # Remove commas like in 123,456
            guess       = float(guess.strip().replace(",", ""))
            scores.append(3.5 if guess == true_answer else -1.5)
        except:
            scores.append(0)
            continue
    return scores

In [67]:
def check_numbers_from_doc(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    # Print only every few steps
    global PRINTED_TIMES
    global PRINT_EVERY_STEPS
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        print(
            '*'*20 + f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}"
        )
    PRINTED_TIMES += 1

    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(-2.5)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            # Remove commas like in 123,456
            guess       = float(guess.strip().replace(",", ""))
            scores.append(3.5 if guess == true_answer else -1.5)
        except:
            scores.append(0)
            continue
    return scores

Warlus operation explanation

In [60]:
import re

# Regex to capture a number (with optional decimal part)
match_numbers = re.compile(r"(-?\d+(?:\.\d+)?)")

responses = [
    "The answer is 42.",
    "I think it's -3.14",
    "No number here!",
    "Maybe 1,234 is correct"
]
extracted_responses = [
    guess.group(1)
    if (guess := match_numbers.search(r)) is not None else None
    for r in responses
]

"""
Warlus operator (`:=`):
does two things at once:
1. Runs match_numbers.search(r)
2. Saves the result into guess
3. Returns the result so it can be used immediately in the expression
"""

print(extracted_responses)


['42', '-3.14', None, '1']


In [61]:
numbers = [1, 2, 3, 4]
result = []
for n in numbers:
    x = n * n
    if x > 5:
        result.append(x)
print(result)

[9, 16]


In [62]:
numbers = [1, 2, 3, 4]

result = [x for n in numbers if (x := n*n) > 5]

print(result)  

[9, 16]


For stable traninig, we want to restrict the model_length
1. remove the top 10% long prompts
2. check the max model input length # it didn't involved in this tutorial

In [63]:
# batch_size = 1000 (default)
tokenized = dataset.map(
    lambda x: {"tokens": tokenizer.apply_chat_template(x['prompt'], add_generation_prompt=True, tokenized=True)}, # add_generation_prompt = True for gettin the max possible length
    batched= True
)

# calculate the length
tokenized = tokenized.map(lambda x: {"L": len(x['tokens'])})

maximum_length = int(np.quantile(tokenized['L'], 0.9))
print("Max Length = ", maximum_length)

# Filter only samples smaller than 90% max length
dataset = dataset.select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

Max Length =  171


### Train the model

In [64]:
# We use GRPOTrainer in here, from-scratch implementation will be another folder

max_prompt_length = maximum_length + 1 # + 1 just in case!
max_completion_length = max_seq_length - max_prompt_length

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

In [70]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    vllm_sampling_params=vllm_sampling_params,
    temperature=1.0,
    learning_rate=5e-6,
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type='linear',
    optim='adamw_8bit',
    logging_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1, # Increase to 4 for smoother training
    num_generations=4,
    max_prompt_length=max_prompt_length,
    max_completion_length=max_completion_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 100,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [76]:
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,  
    reward_funcs = [
        match_format_exactly,
        match_format_partially,
        check_answer,
        check_numbers
    ],
    args=training_args,
    train_dataset=dataset,
    
    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 11,483 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 16,515,072 of 4,038,983,168 (0.41% trained)


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_partially / mean,rewards / match_format_partially / std,rewards / check_answer / mean,rewards / check_answer / std,rewards / check_numbers / mean,rewards / check_numbers / std
1,0.003500,-6.125000,0.750000,357.000000,357.000000,357.000000,1.000000,0.000000,0.000000,0.000000,3.466307,0.000000,0.000000,-2.625000,0.750000,-2.000000,0.000000,-1.500000,0.000000
2,0.007100,-6.500000,0.000000,397.000000,397.000000,397.000000,1.000000,0.000000,0.000000,0.000000,7.079826,0.000000,0.000000,-3.000000,0.000000,-2.000000,0.000000,-1.500000,0.000000
3,0.008800,-6.500000,0.000000,370.000000,370.000000,370.000000,1.000000,0.000000,0.000000,0.000000,8.836608,0.000000,0.000000,-3.000000,0.000000,-2.000000,0.000000,-1.500000,0.000000
4,0.010700,-6.500000,0.000000,428.000000,428.000000,428.000000,1.000000,0.000000,0.000000,0.000000,10.672960,0.000000,0.000000,-3.000000,0.000000,-2.000000,0.000000,-1.500000,0.000000
5,0.009400,-6.750000,0.500000,291.250000,1.000000,388.000000,0.750000,1.000000,1.000000,1.000000,9.358377,0.000000,0.000000,-3.000000,0.000000,-2.000000,0.000000,-1.750000,0.500000
6,0.015500,-6.500000,0.000000,395.000000,395.000000,395.000000,1.000000,0.000000,0.000000,0.000000,15.530737,0.000000,0.000000,-3.000000,0.000000,-2.000000,0.000000,-1.500000,0.000000
7,0.003000,-6.125000,0.750000,375.000000,375.000000,375.000000,1.000000,0.000000,0.000000,0.000000,3.004841,0.000000,0.000000,-2.625000,0.750000,-2.000000,0.000000,-1.500000,0.000000
8,0.011200,-6.500000,0.000000,346.250000,269.000000,372.000000,0.750000,269.000000,269.000000,269.000000,11.200824,0.000000,0.000000,-3.000000,0.000000,-2.000000,0.000000,-1.500000,0.000000
9,0.002500,-6.500000,0.000000,418.000000,418.000000,418.000000,1.000000,0.000000,0.000000,0.000000,2.487236,0.000000,0.000000,-3.000000,0.000000,-2.000000,0.000000,-1.500000,0.000000
10,0.014200,-6.750000,0.500000,312.250000,1.000000,416.000000,0.750000,1.000000,1.000000,1.000000,14.202540,0.000000,0.000000,-3.000000,0.000000,-2.000000,0.000000,-1.750000,0.500000


********************Question:
Acme Corporation has released an alphabet soup in which each of the vowels (A, E, I, O, U) of the English alphabet appears five times (and the consonants do not appear at all). How many five-letter words can be formed from a bowl of Acme Vowel Soup? (Note: The words do not have to be actual words in English!) 
Answer:
3125 
Response:
Okay, so this problem is about forming five-letter words using only the vowels A, E, I, O, U, and each of these vowels can only be used up to five times from the soup. Got it. 

First, let me make sure I understand the constraints:
1. The words are exactly five letters long.
2. The only letters allowed are A, E, I, O, U, and each of these can appear at most five times in the same word.
3. Words don't have to be real English words.

Okay, so since the words are five letters long and we have five different vowels, it feels like permutations problem. Maybe the starting point is to consider if repetition is allowed. Hmm, wait, the

TrainOutput(global_step=100, training_loss=0.006163071701594163, metrics={'train_runtime': 4924.7777, 'train_samples_per_second': 0.081, 'train_steps_per_second': 0.02, 'total_flos': 0.0, 'train_loss': 0.006163071701594163})

Check the prompts and completions used in function: `check_numbers`

In [ ]:
test_prompts = [[{'content': 'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>', 'role': 'system'}, {'content': 'Let $ABCD$ be a unit square in the plane. Points $X$ and $Y$ are chosen independently and uniformly at random on the perimeter of $ABCD$. If the expected value of the area of triangle $\\triangle AXY$ can be expressed as $\\frac{m}{n}$ for relatively prime positive integers $m$ and $n$, compute $m+n$.', 'role': 'user'}], [{'content': 'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>', 'role': 'system'}, {'content': 'Let $ABCD$ be a unit square in the plane. Points $X$ and $Y$ are chosen independently and uniformly at random on the perimeter of $ABCD$. If the expected value of the area of triangle $\\triangle AXY$ can be expressed as $\\frac{m}{n}$ for relatively prime positive integers $m$ and $n$, compute $m+n$.', 'role': 'user'}], [{'content': 'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>', 'role': 'system'}, {'content': 'Let $ABCD$ be a unit square in the plane. Points $X$ and $Y$ are chosen independently and uniformly at random on the perimeter of $ABCD$. If the expected value of the area of triangle $\\triangle AXY$ can be expressed as $\\frac{m}{n}$ for relatively prime positive integers $m$ and $n$, compute $m+n$.', 'role': 'user'}], [{'content': 'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>', 'role': 'system'}, {'content': 'Let $ABCD$ be a unit square in the plane. Points $X$ and $Y$ are chosen independently and uniformly at random on the perimeter of $ABCD$. If the expected value of the area of triangle $\\triangle AXY$ can be expressed as $\\frac{m}{n}$ for relatively prime positive integers $m$ and $n$, compute $m+n$.', 'role': 'user'}]]

In [ ]:
test_completions = [[{'role': 'assistant', 'content': " \nWe start by considering the unit square \\(ABCD\\) with vertices \\(A = (0,0)\\), \\(B = (1,0)\\), \\(C = (1,1)\\), and \\(D = (0,1)\\).\n\nNext, we analyze the problem by breaking it down into manageable parts:\n1. **Perimeter and Coordinates**: We need to consider the perimeter of the square. The perimeter consists of four segments: \\(AB\\), \\(BC\\), \\(CD\\), and \\(DA\\). We parameterize each segment using a variable, for example, for segment \\(AB\\), we can use a parameter \\(t\\) such that \\(0 \\le t \\le 1\\), where \\((x,y) = (t,0)\\).\n2. **General Point Representation**: The points \\(X\\) and \\(Y\\) can be represented on this parameterized perimeter. Let \\(X = (x_1, y_1)\\) and \\(Y = (x_2, y_2)\\).\n3. **Area of Triangle \\(\\triangle AXY\\)**: We need to compute the area of triangle \\(\\triangle AXY\\) for any random points \\(X\\) and \\(Y\\) on the perimeter.\n4. **Expected Value of Area**: We need to find the expected value of the area of \\(\\triangle AXY\\) over all possible positions of \\(X\\) and \\(Y\\). By symmetry, the expected value remains invariant across each segment of the perimeter.\n\nUsing the formula for the area of a triangle with vertices \\((0,0)\\), \\((a,b)\\), and \\((c,d)\\):\n\n\\[\n\\text{Area} = \\frac{1}{2} \\left| ad - bc \\right|\n\\]\n\nWe can apply this formula for our specific squares and derived points.\n\nLet's compute the explicit expected value using"}], [{'role': 'assistant', 'content': "First, let's analyze the problem. We need to find the expected value of the area of triangle $\\triangle AXY$, where $X$ and $Y$ are uniformly and independently chosen on the perimeter of a unit square $ABCD$. \n\nTo solve this, we will consider different cases based on the positions of points $X$ and $Y$ along the perimeter. Points on the perimeter can be described by their coordinates relative to the corners of the square. Let's assume the square is oriented such that $A$ is at $(0,0)$, $B$ is at $(1,0)$, $C$ is at $(1,1)$, and $D$ is at $(0,1)$.\n\nWe will divide the problem into smaller cases based on the location of $X$ and $Y$:\n\n1. Both $X$ and $Y$ are on the same side:\n2. $X$ and $Y$ are on adjacent sides:\n3. $X$ and $Y$ are on opposite sides:\n\nLet's calculate the expected area for each case:\n\n1. Both $X$ and $Y$ on the same side:\n  - Formula for area: $\\frac{1}{2} \\times \\text{base} \\times \\text{height}$\n  - Expected area: $\\frac{1}{4}$ (formed by $\\triangle ABX$ and $\\triangle ADY$)\n  \n2. $X$ and $Y$ on adjacent sides:\n  - To calculate the expected area here, we consider the midpoint as a balancing point. The expected value is the weighted average of areas comparing midpoint to $X$ and $Y$, which would average out over a quarter of the square:\n  - We can see that the expected area for this swapping (split of the diamond shape made) 4 times, giving us $\\frac{1}{"}], [{'role': 'assistant', 'content': ' <SOLUTION>51</SOLUTION>'}], [{'role': 'assistant', 'content': "Oh dear - what a messy one for you. I might start by drawing a picture (while remembering that a unit square has all sides of unit length).\n\nHere is a good Aops discussion of the topic: https://artofproblemsolving.com/wiki/index.php?title=2001_AIME_I_Problems/Problem_9 (the problem could be restated here as, Essentially, it says that two points on the perimeter are chosen and we want the average area of the triangle connecting them with $(0,0)$. This questions assumes the area is evenly 'spread' along the triangular 'space' ie 2/3 of it can define the solutions. So the answer should be 1*1/3)</START_WORKING_OUT><SOLUTION>The area of the interior of the square bounded by the triangle $AXY$ is \n\\[\\iint_{\\square ABCD} x = \\frac{1}{4} \\iint_{\\square 0001} xy\\,dx\\, dy = \\frac{1}{4},\\]so expect \\[\\boxed02.\\]</SOLUTION>\n\nReference: https://web2.0calc.com/questions/this-is-an-imo-geometry-problem-2022-july"}]]

In [ ]:
test_prompts[0][0]

{'content': 'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>',
 'role': 'system'}

In [ ]:
test_sys_dict = {}
test_user_dict = {}
for i in range(len(test_prompts)):
    for j in range(i, len(test_prompts), 1):
        test_sys_dict[str(test_prompts[i][0])] = str(test_prompts[j][0])
        test_user_dict[str(test_prompts[i][1])] = str(test_prompts[j][1])

In [ ]:
def check_data(input_list: list):
    test_role_dict = {}
    test_content_dict = {}
    for i in range(len(input_list)):
        for j in range(i, len(input_list), 1):
            test_role_dict[str(input_list[i][0])] = str(input_list[j][0])
            test_content_dict[str(input_list[i][-1])] = str(input_list[j][-1])

    print(f"num of unique {input_list[0][0]['role']}: {len(test_role_dict.keys())}")
    print(f"num of unique content: {len(test_content_dict.keys())}")

In [ ]:
check_data(test_prompts)

num of unique system: 1
num of unique content: 1


In [ ]:
check_data(test_completions)

num of unique assistant: 4
num of unique content: 4


- The prompts are duplicate
- There are `num_generation` different completions 